## Adventures with Elephants

Description of the dataset:
---------------------------

This dataset has been recorded Angela S. Stoeger (angela.stoeger-horwath@univie.ac.at) from Vienna University and colleagues. Acoustic recordings were collected from three female and two male African elephants (Loxodonta africana) aged between 9 and 17 years located at Adventures with Elephants, Bela Bela, South Africa in 2011. The elephants were fully habituated to human presence and free to roam around in a savannah reserve of 300 ha. Vocalizations were recorded during two social contexts: spatial separation and subsequent bondings, recording distances between the elephant and the microphone ranged from 10 to several hundreds of meters. 
In 359 minutes of recordings we annotated 635 rumbles by manually tagging the beginning and the end of each rumble.

In [1]:
# annotations are in matlab file, which apparently is a hdf5 format
import scipy.io as sio

filepath = './AdventuresWithElephants_Dataset/anno/groundTruth.mat'

annos = sio.loadmat(filepath)
data = annos['variable']

In [2]:
# r=[(x['SHAPE@XY'][i][0],x['SHAPE@XY'][i][1],x['PrimeKey'][i]) for i in range(x.shape[0])]
# unpack nd array
u = [(data[i,0][0],data[i,1][0][0],data[i,2][0][0],data[i,3][0]) for i in range(1,data.shape[0])]

In [3]:
# build dataframe from list
import pandas as pd
df = pd.DataFrame(data=u)  # 1st row as the column names
df.columns=['filename','start','duration','type']

Data format:
------------

Folder "wavs" contains all recorded signals (sampled at 44100Hz, 16bit, mono)

Folder "anno" contains the annotation in binary Matlab format (*.mat). The annotations include:

- groundTruth.mat: a cell that contains one annotated rumble in each row. The first column provides the name of the wav-file. The second column gives the start position of the rumble in samples; the third column contains the length of the annotated rumble in samples. The fourth column contains the type of rumble. There are two types (the usual "rumble" and the "snumble", which is a mixture of snort and rumble).

In [4]:
df['type'].value_counts()

rumble     624
snumble     11
Name: type, dtype: int64

In [5]:
# issue with EFAF2011A011.WAV en 012.WAV
df = df[df['filename'] != "EFAF2011A011.WAV"]
df = df[df['filename'] != "EFAF2011A012.WAV"]
df = df.reset_index(drop=True)

In [6]:
# Code die aangeeft of de 10 seconden ervoor background is (1=ja, 0=nee)
df["background"] = 1
sr_target = 44100
# als (start-sr > 0) dan 0
df.loc[df["start"] < sr_target, "background"] = 0

In [7]:
for i in range(0, df.shape[0]):
    # skip 1e rij
    if(i==0):
        df.loc[i, "background"] = 0
    else:
        # Bereken eindtijd van vorige rij
        previous_end = df.loc[i-1, "start"] + df.loc[i-1, "duration"]
        # Check of filename zelfde is als file vorige rij
        if(df.loc[i-1, "filename"] == df.loc[i, "filename"]):
            # Check of previous end + sr_target*10 > start (dan 0)
            if(previous_end + sr_target*10 > df.loc[i, "start"]):
                #df.at[i,'ifor']
                df.loc[i, "background"] = 0
        else:
            df.loc[i, "background"] = 0

In [8]:
df['background'].value_counts()

0    307
1    305
Name: background, dtype: int64

In [9]:
df.iloc[210:213,]

,filename,start,duration,type,background
210,EFAF2011A022.WAV,33176566,213885,rumble,0
211,EFAF2011A022.WAV,33398508,97422,rumble,0
212,EFAF2011A023.WAV,219626,102431,rumble,0


In [10]:
# now we open the source file, cut out the piece that is annotated and save that as a 5 second sample
import sys, csv, os, shutil
import numpy as np

# we will also resample the original recordings
#import librosa
import soundfile as sf
import resampy

from tqdm import tqdm

# this is the sample reate we want
sr_target = 44100

# source dataset location
source_path = './AdventuresWithElephants_Dataset/wavs/all'
dest_path = "./AdventuresWithElephants_Dataset/wavs/samples/"

fileList = []

# for each annotation do
for index,row in tqdm(df.iterrows(), total=len(df)):
    print("index: " + str(index))
    print("row: ")
    print(row)
    # map mat file contents
    source_filename = row.filename
    start = row.start
    duration = row.duration
    category = row.type
    print("source filename: " + source_filename)
    # read source wav 
    src_file_path = os.path.normcase("%s/%s" % (source_path,source_filename))
    print(src_file_path)
    # check if sourcefile exists
    if os.path.exists(src_file_path):
        # resample
        # Load audio file at its native sampling rate
        #x, sr_orig = librosa.load(src_file_path, mono=True, sr=None, dtype=np.int16)
        x, sr_orig = sf.read(src_file_path, dtype='int16')
        print("x: ")
        print(x)
        print("sr_orig: " + str(sr_orig))
        # We can resample this to any sampling rate we like, say 16000 Hz
        # define 5 seconds
        s5 = 10 * sr_orig
        # clip from start to next 5 seconds resample in target sample rate
        y = resampy.resample(x[start:start+s5+1], sr_orig, sr_target)
        print("y: ")
        print(y)
        # check if previous 5 secs is background
        # if so create a background sample
        if (row.background == 1):
            # clip from start to next 5 seconds resample in target sample rate
            yb = np.int16(resampy.resample(x[start-s5:start+1], sr_orig, sr_target))
            print("yb class: " + str(yb))
            # make destination path
            b_dst_fname = "Background" + source_filename.split('.')[0] + '-' + str(round((start-s5)/sr_orig)) + '.' + source_filename.split('.')[1]
            # add meta data to list
            fileList.append([b_dst_fname,'background'])
            # path
            b_dst_file_path = "%s/%s" % (dest_path,b_dst_fname)
            # write it back
            #librosa.output.write_wav(b_dst_file_path, yb, sr_target)
            sf.write(b_dst_file_path, yb, sr_target)
        # make destination path
        dest_filename = "Elephant" + source_filename.split('.')[0] + '-' + str(round(start/sr_orig)) + '.' + source_filename.split('.')[1]
        dst_file_path = "%s/%s" % (dest_path,dest_filename)

        # create meta data file
        fileList.append([dest_filename,category])

        # write it back
        #librosa.output.write_wav(dst_file_path, y, sr_target)
        sf.write(dst_file_path, y, sr_target)
        #break;
    else:
        print('File %s not found.' % source_filename)
        break;

  0%|                                                                                          | 0/612 [00:00<?, ?it/s]

index: 0
row: 
filename      EFAF2011A008.WAV
start                  1688633
duration                 51548
type                    rumble
background                   0
Name: 0, dtype: object
source filename: EFAF2011A008.WAV
.\adventureswithelephants_dataset\wavs\all\efaf2011a008.wav
x: 
[-53 -16 -15 ...  76  72  59]
sr_orig: 44100
y: 
[   0    0    0 ... -572 -611 -597]


  0%|▏                                                                                 | 1/612 [00:01<10:19,  1.01s/it]

index: 1
row: 
filename      EFAF2011A008.WAV
start                  1829197
duration                 24136
type                    rumble
background                   0
Name: 1, dtype: object
source filename: EFAF2011A008.WAV
.\adventureswithelephants_dataset\wavs\all\efaf2011a008.wav
x: 
[-53 -16 -15 ...  76  72  59]
sr_orig: 44100
y: 
[ 4  0 -7 ...  6  0  0]


  0%|▎                                                                                 | 2/612 [00:01<07:12,  1.41it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████▋| 610/612 [06:16<00:01,  1.62it/s]

index: 610
row: 
filename      EFAF2011A033.WAV
start                 28492211
duration                112274
type                    rumble
background                   1
Name: 610, dtype: object
source filename: EFAF2011A033.WAV
.\adventureswithelephants_dataset\wavs\all\efaf2011a033.wav
x: 
[ 19  22  25 ...  56 -23 -79]
sr_orig: 44100
y: 
[-50 -48 -44 ...  30  36  32]
yb class: [ 25  31  28 ... -42 -50 -50]


100%|███████████████████████████████████████████████████████████████████████████████▊| 611/612 [06:17<00:00,  1.62it/s]

index: 611
row: 
filename      EFAF2011A033.WAV
start                 28575662
duration                 95582
type                    rumble
background                   0
Name: 611, dtype: object
source filename: EFAF2011A033.WAV
.\adventureswithelephants_dataset\wavs\all\efaf2011a033.wav
x: 
[ 19  22  25 ...  56 -23 -79]
sr_orig: 44100
y: 
[16 20 18 ...  0  0  0]


100%|████████████████████████████████████████████████████████████████████████████████| 612/612 [06:18<00:00,  1.62it/s]


In [11]:
# save the metadata for later
fileList
md = pd.DataFrame(fileList, columns=["filename", "category"])  
md.to_csv('./dataset/meta-data-elephant-adventures.csv',index=False)

In [12]:
md.tail()

,filename,category
912,BackgroundEFAF2011A033-417.WAV,background
913,ElephantEFAF2011A033-427.WAV,rumble
914,BackgroundEFAF2011A033-636.WAV,background
915,ElephantEFAF2011A033-646.WAV,rumble
916,ElephantEFAF2011A033-648.WAV,rumble


In [13]:
# copy all wavs to audio dir in serval
import glob
import shutil
import os

src_dir = "../../data/AdventuresWithElephants_Dataset/wavs/samples/"
dst_dir = "./dataset/audio/"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.WAV")):
    shutil.copy(jpgfile, dst_dir)
